In [ ]:
from keras.layers import Convolution2D,MaxPooling2D,Dropout,Dense,Flatten,GlobalAveragePooling2D
from keras import models,losses,optimizers
from keras.preprocessing.image import ImageDataGenerator
from keras.applications import ResNet50
from keras.models import Sequential

In [ ]:
base_model = applications.resnet50.ResNet50(weights= 'imagenet', include_top=False, input_shape= (48,48,3))
model = Sequential()
model.add(base_model)
model.add(Flatten())
model.add(Dense(256, activation='relu', name='Dense_Intermediate'))
model.add(Dropout(0.1, name='Dropout_Regularization'))
model.add(Dense(24, activation='sigmoid', name='Output'))

In [ ]:
for cnn_block_layer in model3.layers[0].layers:
    cnn_block_layer.trainable = False
model3.layers[0].trainable = False